# Bullying recognition from Tweeter Corpus, build on recent events

### 1. First snippet of code reads the file, which containes the tweets in json fromat, gatherered by streiming the tweeter  

In [9]:
# Load libraries
import pandas as pd  
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import requests

In [10]:
import json
import difflib
import copy
from pprint import pprint
import csv
import numpy as np

#Reading the file
f = open('pythontest1.json')
with open('pythontest1.json',encoding='utf-8') as f:
    content = f.readlines()
#Getting rid of empty strings
content =  [x.strip() for x in content]
content = list(filter(None, content)) 
TweetsJsonArray = []
#To JSON TYPE
for tweet in content:
    TweetsJsonArray.append(json.loads(tweet))  

### 1.1 Loading Dictionaries

In [11]:
collectivenouns=[]
pronouns=[]
offensiveWords = dict() 
xl = pd.ExcelFile("Dictionary.xlsx")

df = xl.parse("OffensiveWords")
l = len(df)
for i in range(l)[1:]:
    offensiveWords[(df['Word'][i]).lower()]=df['Weight'][i]
    
df = xl.parse("Nouns")
l = len(df)
for i in range(l)[1:]:
    collectivenouns.append((df['Collective Nouns'][i]).lower()) 
    
df = xl.parse("Pronouns")
l = len(df)
for i in range(l)[1:]:
    pronouns.append(df['Personal Pronouns'][i])

# reading the bad words file 
with open('bad-words.csv') as csvfile:    
    readCSV = csv.reader(csvfile, delimiter=',')
    badwords = []
    weight = []
    count = []
    result = []
    for row in readCSV:       
        badwords.append(row[0])
        weight.append(int(row[1]))
#Reading "Bad words" from the file
with open('nouns.csv', newline='') as csvfile:
    readCSV1 = csv.reader(csvfile, delimiter=',')
    nouns=[]
    for row in readCSV1:
        nouns.append(row[0].lower())


### 2. The code snippet below contains the functions, used accross the project, such as the Offensiveness level calculation, two tweets similarity calculation 

In [12]:
#Preprocessing - deleting the similar tweets- deciede that it is not needed
import difflib
def similar(a, b):
    return difflib.SequenceMatcher(None, a, b).ratio()
#Calculating the offinsiveness level

def offensiveness_weight(TweetText):        
        templist = TweetText.split()
        offensiveWordsList=list(offensiveWords.keys())
        temp = set(offensiveWordsList).intersection(templist)
        my_list = list(temp)        
        final = 0
        result=0
        
        if len(my_list)!=0:
            for word in my_list:                
                final = float(final) + float(offensiveWords[word])        
            result = final / len(my_list)
        return result
    
#Identifying the existance of the words

def wordpresence(TweetText,wordlist):
    templist1= TweetText.split()    
    temp1 = set(wordlist).intersection(templist1) 
    my_list1 = list(temp1)    
    if len(my_list1)!=0:
        return 1
    else:
        return 0
# If 
def offensive_level(TweetText):
    
    nounsExistance = wordpresence(TweetText,collectivenouns)    
    pronounsExistance = wordpresence(TweetText,pronouns)
    
    if offensiveness_weight(TweetText) > 0 and (wordpresence or pronounsExistance) :    
        return (offensiveness_weight(TweetText) + 1)/6 * 100
    else:
        return (offensiveness_weight(TweetText))/6 * 100
#Count how many words from the wordlist parameter are in tweet
def WordCount(TweetText,wordlist):
    templist1= TweetText.split()    
    temp1 = set(wordlist).intersection(templist1) 
    my_list1 = list(temp1)     
    return len(my_list1)


### 2.2 Third Party Libriries

In [13]:
import requests
def SentimentalAnalysis(tweet):
    r = requests.post("http://text-processing.com/api/sentiment/", data={'text': tweet})
    label= r.json()['label']
    if label =='neutral':
        return 0
    elif label=='positive':
        return 1
    else :        
        return 2          

## 3. Preprocessing

In [14]:
#Writing the content to the exel for Manual Analysis
import re
import csv
import json
import xlwt
import re

num=1;
book = xlwt.Workbook()
sheet1 = book.add_sheet("Sheet1",cell_overwrite_ok=True)

row = sheet1.row(0)

#row.write(0, 'Created at')
row.write(1, 'Text')  
row.write(2, 'Names') 
row.write(3, 'Description') 
row.write(4, 'Locations') 
row.write(5, 'Id') 
row.write(6, 'Offensiveness Weight')
row.write(7, 'Offensiveness Level')
row.write(8, 'Offensive Words Amount')
#row.write(9, 'Collective Nouns Amount')
#row.write(10,'Personal Pronouns Amount')
row.write(9,'Sentimental Analysis')
row.write(10,'Class')

n=0
for tweet in TweetsJsonArray: 
    n=n+1
    if tweet['lang']=="en":
        row = sheet1.row(num)
        row.write(0, tweet['created_at'])  
        text=''
        if 'extended_tweet' in tweet:
            text=tweet['extended_tweet']['full_text']
        elif 'retweeted_status' in tweet:
            if 'extended_tweet' in tweet['retweeted_status']:
                text=tweet['retweeted_status']['extended_tweet']['full_text']
            else:
                text=tweet['retweeted_status']['text']
        else:
            text=tweet['text']
        text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+','',text)
        names = tweet['user']['name']
        description = tweet['user']['description']
        #text = re.sub(r'\W+', '', text)
        
        PERMITTED_CHARS = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ_- "
        text = "".join(c for c in text if c in PERMITTED_CHARS)
        text=text.lower()
        text=text.strip()
        names="".join(c for c in names if c in PERMITTED_CHARS)
        description="".join(c for c in description if c in PERMITTED_CHARS) if description!= None else ''
        #text=text+";\n"+description       
        num=num+1              
        
        row.write(1, text)
        row.write(2, names)
        row.write(3, description)
        row.write(4, tweet['user']['location'] if tweet['user']['location'] != None else '')
        row.write(5, num)      
        num=num+1
        
book.save("TweetCorpusExel3.xls")
    #csvFile.writerow([tweet["created_at"],tweet["text"],tweet["user"]["name"],tweet["user"]["description"],tweet["user"]["location"]])

### 3.1 Sort the tweets in the file

In [15]:
xl = pd.ExcelFile("TweetCorpusExel3.xls")
df = xl.parse("Sheet1")
df = df.sort_values(by="Text")
writer = pd.ExcelWriter('TweetCorpusExel4.xlsx')
df.to_excel(writer,'Sheet1',index=False)
writer.save()
writer.close()

### 3.2 Delete repeating tweets

In [16]:
df = pd.read_excel('TweetCorpusExel4.xlsx', sheetname='Sheet1')
writer = pd.ExcelWriter('TweetCorpusExel5.xlsx')
currentrow=0
if str(df['Description'][currentrow])!='nan':
    df['Text'][currentrow]=str(df['Text'][currentrow])+" \n <::> "+str(df['Description'][currentrow])
tweet=str(df['Text'][currentrow])

l = len(df)

for i in range(l)[1:]:   
    if df['Text'][i]==tweet or similar(df['Text'][i],tweet)>0.8:        
        if len(df['Text'][i])>len(tweet):
            df['Text'][currentrow]=df['Text'][i]             
        df['Text'][i]=''        
        df['Id'][currentrow]=str(df['Id'][currentrow])+","+str(df['Id'][i])
        if df['Names'][i]!='':
            df['Names'][currentrow]=str(df['Names'][currentrow]) +","+str(df['Names'][i]) 
        if str(df['Locations'][i])!='nan':
            df['Locations'][currentrow]=str(df['Locations'][currentrow])+","+str(df['Locations'][i])
        
        if str(df['Description'][i])!='nan':
               df['Text'][currentrow]=str(df['Text'][currentrow])+" \n <::> "+str(df['Description'][i])
    else:
        tweet=str(df['Text'][currentrow])
        df['Offensiveness Weight'][currentrow]=offensiveness_weight(tweet)        
        df['Offensiveness Level'][currentrow]=offensive_level(tweet) 
        df['Offensive Words Amount'][currentrow]=WordCount(tweet,offensiveWords)
        df['Sentimental Analysis'][currentrow]=SentimentalAnalysis(tweet)
        currentrow=i        
        tweet=str(df['Text'][currentrow])               
                      
df = df[df.Text != '']   
df.to_excel(writer,'Sheet1',index=False)
writer.save()
writer.close()



/root/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [17]:
df = pd.read_excel('TweetCorpusExel5.xlsx', sheetname='Sheet1')
writer = pd.ExcelWriter('TweetCorpusExelML.xlsx')
del df['Description']

del df['Names']
del df['Locations']
del df['Id']
df.to_excel(writer,'Sheet1',index=False)
writer.save()
writer.close()

In [19]:
df = pd.read_excel('TweetCorpusExelML.xlsx', sheetname='Sheet1') 
writer = pd.ExcelWriter('processed.xlsx')
#df['O_Weight']=df['O_Weight'].fillna(999)
#df['O_level']=df['O_level'].fillna(999)
df=df.dropna() # drops rows with Nan
#cols = ['Offensiveness Level', 'Offensive Words Amount','Offensiveness Weight','Sentimental Analysis', 'Class']
#df[cols] = df[cols].applymap(np.int64)

del df['Text']

df.to_excel(writer,'Sheet1',index=False)
writer.save()
writer.close()

In [20]:

dataset = pd.read_excel(open('processed.xlsx','rb'), sheetname='Sheet1')

In [21]:
print(dataset.head(20))

    Offensiveness Weight  Offensiveness Level  Offensive Words Amount  \
0                    0.0             0.000000                       0   
1                    1.5            41.666667                       2   
2                    1.0            33.333333                       1   
3                    0.0             0.000000                       0   
4                    1.0            33.333333                       1   
5                    0.0             0.000000                       0   
6                    0.0             0.000000                       0   
7                    0.0             0.000000                       0   
8                    0.0             0.000000                       0   
9                    0.0             0.000000                       0   
10                   0.0             0.000000                       0   
11                   2.5            58.333333                       2   
12                   3.0            66.666667      

In [22]:
print(dataset.shape)
print(dataset.describe())
print(dataset.groupby('class').size())

(332, 5)
       Offensiveness Weight  Offensiveness Level  Offensive Words Amount  \
count            332.000000           332.000000              332.000000   
mean               0.966616            23.138387                0.578313   
std                1.408890            30.509416                0.813340   
min                0.000000             0.000000                0.000000   
25%                0.000000             0.000000                0.000000   
50%                0.000000             0.000000                0.000000   
75%                2.000000            50.000000                1.000000   
max                5.000000           100.000000                5.000000   

       Sentimental Analysis       Class  
count            332.000000  332.000000  
mean               0.518072    0.460843  
std                0.877534    0.499217  
min                0.000000    0.000000  
25%                0.000000    0.000000  
50%                0.000000    0.000000  
75%         

KeyError: 'class'

In [23]:
array = dataset.values
X = array[:,0:4]
Y = array[:,4]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [24]:
# Test options and evaluation metric
seed = 7
scoring = 'accuracy'


In [25]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.799858 (0.054046)
LDA: 0.799858 (0.054046)
KNN: 0.776781 (0.104415)
CART: 0.792308 (0.048849)
NB: 0.784758 (0.066335)
SVM: 0.799858 (0.054046)


In [ ]:
print (wordpresence('sweden is the worlds best country for immigrant us studysurprising'))

In [ ]:
similar('ad','ads')

In [ ]:
def EmotionsAnalysis(tweet):
    
    headers={
    "X-Mashape-Key": "9zdoXpyV3Jmshr0ehYsqOnCT6i3np1F4iMqjsnq4mEJLTej82W",
    "Accept": "application/json",
    "Content-Type": "application/x-www-form-urlencoded"
    }
    params={
    "lang": "en",
    "text": "a lot of us wont debate with liberalspro-islamlgbtanti-christians but just because were quiet dont misunderstand our intentions"}
    r = requests.post("https://shl-mp.p.mashape.com/webresources/jammin/emotionV2",headers=headers,params=params)
    jsonObject= r.json()
    return jsonObject

In [ ]:
jsonObject=EmotionsAnalysis('a lot of us wont debate with liberalspro-islamlgbtanti-christians but just because were quiet dont misunderstand our intentions')
print(jsonObject)

### Updating classes

In [ ]:
df = pd.read_excel('TweetCorpusExelMLV.xlsx', sheetname='Sheet1')
df2 = pd.read_excel('TweetCorpusExel6.xlsx', sheetname='Sheet1')

l = len(df)
for i in range(l)[1:]: 
    tweet=df['Class'][i]
    df2[df2['A'].str.contains("tweet")==True]

    
writer.save()
writer.close()